**Functional redundancy and resilience**

Examination of functional redundancy and reslience in the context of microbial biodiversity measurements.
1. compare taxonomic and functional diversity in these same samples to evaluate _functional redundancy_ in microbial communities.
2. examine longitudinal change in these biodiversity metrics to measure _resilience_ in microbial communities

- Metadata : sample_meta_data (done in A)
- Diversity analysis : alpha = core-metrics-results (done in F), beta = core-metrics-results-bd (done in G)
- Metagenome content predicted by PICRUST2 : (done in I)


<a id='setup'></a>
## 0. Setup

In [3]:
import os
import qiime2 as q2
import pandas as pd
from qiime2 import Visualization



# do not increase this value!
n_jobs = 3
    
%matplotlib inline

In [4]:
data_dir ='project_data'

In [11]:
md = q2.Metadata.load(data_dir + '/sample_meta_data.tsv').to_dataframe()
pd.DataFrame([str(sorted(md[col].astype(str).unique())) for col in md.columns],
             index=pd.Index(md.columns, name='Column'), columns=['Values'])

,Values
Column,
GEN_age_cat,"['20s', '30s', '40s', '50s', '60s', '70+', 'No..."
GEN_age_corrected,"['11.0', '14.0', '15.0', '16.0', '17.0', '18.0..."
GEN_bmi_cat,"['Normal', 'Not provided', 'Obese', 'Overweigh..."
GEN_bmi_corrected,"['11.2', '11.57', '14.34', '14.62', '14.79', '..."
GEN_cat,"['False', 'True']"
GEN_collection_timestamp,"['2014-05-10 12:00:00', '2015-01-01 09:00:00',..."
GEN_country,"['Australia', 'Belgium', 'Canada', 'Georgia', ..."
GEN_dog,"['False', 'True']"
GEN_elevation,"['-0.9', '0.0', '1.4', '10.4', '10.8', '10.9',..."


<a id='function'></a>
## 1.a Functional redundancy calculation

Next we will look at predicted gene pathway information to compare taxonomic vs. functional diversity patterns. 
We will use the `core-metrics` pipeline on the `pathway_abundance.qza` table, which consists of PICRUST2-predicted gene pathway counts. 

In [5]:
! qiime diversity core-metrics \
  --i-table $data_dir/pathway_abundance.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-sampling-depth 100000 \
  --p-n-jobs $n_jobs \
  --output-dir $data_dir/core-metrics-picrust2  

Saved FeatureTable[Frequency] to: project_data/core-metrics-picrust2/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: project_data/core-metrics-picrust2/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: project_data/core-metrics-picrust2/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: project_data/core-metrics-picrust2/evenness_vector.qza
Saved DistanceMatrix to: project_data/core-metrics-picrust2/jaccard_distance_matrix.qza
Saved DistanceMatrix to: project_data/core-metrics-picrust2/bray_curtis_distance_matrix.qza
Saved PCoAResults to: project_data/core-metrics-picrust2/jaccard_pcoa_results.qza
Saved PCoAResults to: project_data/core-metrics-picrust2/bray_curtis_pcoa_results.qza
Saved Visualization to: project_data/core-metrics-picrust2/jaccard_emperor.qzv
Saved Visualization to: project_data/core-metrics-picrust2/bray_curtis_emperor.qzv


## Hypothesis 1 : Drinks

In [5]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_alcohol_frequency*NUT_drinks_per_session" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h1.qzv

Saved Visualization to: project_data/core-metrics-picrust2/adonis-bc-h1.qzv


In [6]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h1.qzv')

<visualization: Visualization uuid: c1ed3309-a9a3-42aa-b7bd-4fea8827bfd1>

## Hypothesis 2 : Milk products

In [7]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_milk_cheese_frequency*NUT_milk_substitute_frequency*NUT_vitamin_d_supplement_frequency" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h2.qzv

Saved Visualization to: project_data/core-metrics-picrust2/adonis-bc-h2.qzv


In [8]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h2.qzv')

<visualization: Visualization uuid: c08c71aa-2647-43ad-87d8-316acc42f62f>

## Hypothesis 3 : Mediterranean

In [9]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_olive_oil*NUT_seafood_frequency*NUT_vegetable_frequency*NUT_fruit_frequency*NUT_whole_grain_frequency" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h3.qzv

Saved Visualization to: project_data/core-metrics-picrust2/adonis-bc-h3.qzv


In [10]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h3.qzv')

<visualization: Visualization uuid: 744446ff-8f61-4d5f-8775-ef42c5e756ca>

## Hypothesis 4 : Poultry & Meat

In [11]:
! qiime diversity adonis \
  --i-distance-matrix $data_dir/core-metrics-picrust2/bray_curtis_distance_matrix.qza \
  --m-metadata-file $data_dir/sample_meta_data.tsv \
  --p-formula "NUT_whole_eggs*NUT_poultry_frequencyNUT_high_fat_red_meat_frequency*NUT_meat_eggs_frequency*NUT_red_meat_frequency*NUT_vitamin_b_supplement_frequency" \
--o-visualization $data_dir/core-metrics-picrust2/adonis-bc-h4.qzv

Plugin error from diversity:

  'NUT_poultry_frequencyNUT_high_fat_red_meat_frequency' is not a column in the metadata. Available columns: 'GEN_age_cat', 'GEN_age_corrected', 'GEN_bmi_cat', 'GEN_bmi_corrected', 'GEN_cat', 'GEN_collection_timestamp', 'GEN_country', 'GEN_dog', 'GEN_elevation', 'GEN_geo_loc_name', 'GEN_height_cm', 'GEN_host_common_name', 'GEN_last_move', 'GEN_last_travel', 'GEN_latitude', 'GEN_level_of_education', 'GEN_longitude', 'GEN_race', 'GEN_sample_type', 'GEN_sex', 'GEN_weight_kg', 'NUT_alcohol_consumption', 'NUT_alcohol_frequency', 'NUT_alcohol_types_beercider', 'NUT_alcohol_types_red_wine', 'NUT_alcohol_types_sour_beers', 'NUT_alcohol_types_spiritshard_alcohol', 'NUT_alcohol_types_unspecified', 'NUT_alcohol_types_white_wine', 'NUT_artificial_sweeteners', 'NUT_diet_type', 'NUT_drinks_per_session', 'NUT_fed_as_infant', 'NUT_fermented_plant_frequency', 'NUT_frozen_dessert_frequency', 'NUT_fruit_frequency', 'NUT_gluten', 'NUT_high_fat_red_meat_frequency', 'NUT_homeco

In [12]:
Visualization.load(f'{data_dir}/core-metrics-picrust2/adonis-bc-h4.qzv')

<visualization: Visualization uuid: d11b9012-6396-4187-adcd-c9f0e15c7b2e>